![](img/logo.png)

# Data analysis: Pandas and Seaborn

__Pandas__ is a very strong library for manipulating large and complex datasets using a new data structure, the **data frame**, which models a table of data.

Pandas helps to close the gap between Python and R for data analysis and statistical computing.

Pandas data frames address three deficiencies of NumPy arrays:

- data frames hold heterogenous data; each column can have its own numpy.dtype,
- the axes of a data frame are labeled with column names and row indices,
- and, they account for missing values which this is not directly supported by arrays.

Data frames are extremely useful for data manipulation.
They provide a large range of operations such as filter, join, and group-by aggregation, as well as plotting.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print('Pandas version:', pd.__version__)

# Statistical Analysis of Life History Traits

We will analyze animal life-history data from [AnAge](http://genomics.senescence.info/download.html#anage). 

In [ ]:
data = pd.read_csv('../data/anage_data.txt', sep='\t') # lots of other pd.read_... functions
print(type(data))
print(data.shape)

Pandas holds data in `DataFrame` (similar to __R__).

`DataFrame` have a single row per observation (in contrast to the previous exercise in which each table cell has one observation), and each column has a single variable. 

Variables can be numbers or strings.

The `head` method gives us the 5 first rows of the data frame.

In [ ]:
data.head()

The column attribute holds the column names

In [ ]:
print(data.columns)

In [ ]:
data.columns[:3]

We can select a specific column:

(note that the result is a view, not a new object!)

In [ ]:
bodyCol = data['Body mass (g)']
print(type(bodyCol))
print(len(bodyCol))

We can look for the min / max value in a column

In [ ]:
# general min / max functions
print(min(bodyCol))
print(max(bodyCol))

# pandas min/max - ignores missing values (nan)
print(bodyCol.min())
print(bodyCol.max())

We can examine how many unique values there are in a specific column:

In [ ]:
data['Kingdom'].unique()

In [ ]:
data['Phylum'].unique()

In [ ]:
data.columns[1:8]

In [ ]:
for elem in data.columns[1:8]:
    print('Unique {}:\t{}'.format(elem, len(data[elem].unique())))

pandas `DataFrame` allows richer indexing.

For example, let's browse our data for species that have body mass greater than 300 kg.

First we will a create new column (`Series` object) that tells us if a row is a large animal row or not:

In [ ]:
large_index = data['Body mass (g)'] > 300 * 1000 # 300 kg
large_index.head()

In [ ]:
print(large_index.shape)
print(large_index.any())
print(large_index.sum())

Now, we can slice our data with this boolean index.

In [ ]:
large_data = data[large_index] # Note: this is not a new objec!
large_data

The `iterrows` method let's us iterate over the rows of the data.

For each row we get both the row as a `Series` object (similar to `dict` for our use) and the row number as an `int` (this is similar to the use of `enumerate` on lists and strings).

In [ ]:
for i, row in large_data.iterrows(): 
    print(row['Common name'], row['Body mass (g)']/1000, 'kg')

So... a [Dromedary](http://en.wikipedia.org/wiki/Dromedary) is the single-humped camel.

![Camel](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Camelus_dromedarius_on_Sinai.jpg/220px-Camelus_dromedarius_on_Sinai.jpg)

In [ ]:
data['Body mass (g)'][:3]

By looking at the first 3 measurements we see that there are quite a few missing values. 

NaN are slots that were empty in the data file.

NaN values can be treated in many ways, one straightforward solution is simply remove these lines.

In [ ]:
clean = data.dropna(subset = ['Body mass (g)', 'Metabolic rate (W)'])
# drops a line if nan values is in subset

clean.shape

In [ ]:
#clean.hist('Body mass (g)');
print((clean['Body mass (g)'] >  10 * 1000).sum())
print((clean['Body mass (g)'] <  10 * 1000).sum())

Let's continue with small and medium animals - we filter out anything that has body mass of more than 10 kg.

In [ ]:
data = data[data['Body mass (g)'] <  10 * 1000] 
data.shape

For starters, let's plot a scatter of body mass vs. metabolic rate.

Because we work with pandas, we can do that with the `plot` method of `DataFrame`, specifying the columns for `x` and `y` and a plotting style (without the style we would get a line plot which makes no sense here).

In [ ]:
data.plot.scatter(x='Body mass (g)', y='Metabolic rate (W)')
plt.xlim(0,None)
plt.ylim(0,None);
#plt.xscale('log')
#plt.yscale('log');

If this plot looks funny, you are probably using Pandas with version <0.22.

From this plot it seems that:
1. there is a correlation between body mass and metabolic rate, and 
1. there are many small animals (less than 2 kg) and not many medium animals (between 5 and 10 kg).

Before we continue, let's add another column that will show the mass in kg:

In [ ]:
data['Body mass (kg)'] = data['Body mass (g)'] / 1000

Next, let's check how many records do we have for each Class (as in the taxonomic unit): 

In [ ]:
class_counts = data['Class'].value_counts()
print(class_counts)

In [ ]:
class_counts['Mammalia']+class_counts['Aves']

In [ ]:
class_counts.plot.bar()
plt.ylabel('Num. of species');

So we have lots of mammals and birds, and a few reptiles and amphibians. 

This is important as amphibian and reptiles could have a different replationship between mass and metabolism because they are cold blooded.

## Exercise: data frames

1) **Print the number** of reptiles in this dataset, and how many of them are of the genus `Python`.

In [ ]:
reptiles = 

pythons = 

In [ ]:
print("# of reptiles: ", reptiles)
print("# of pythons: ", pythons)

2) **Plot the histogram of the mammal body masses** using `plot.hist()`.

Since most mammals are small, the histogram looks better if we plot a cumulative distribution rather then the distribution - we can do this with the `cumulative` argument. You also need to specify a higher `bins` argument then the default.

# Seaborn

Let's do a simple linear regression plot; but let's do it in separate for each Class. 

We can do this kind of thing with Matplotlib and [SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html), but a very good tool for statistical visualizations is **[Seaborn](http://seaborn.pydata.org)**.

Seaborn adds on top of Pandas a set of sophisticated statistical visualizations, similar to [ggplot2](http://ggplot2.org) for R.

In [ ]:
import seaborn as sns
sns.set_context("talk") # thist sets figure settings such as size of various elements

In [ ]:
ax = sns.lmplot(
    x='Body mass (kg)', 
    y='Metabolic rate (W)', 
    hue='Class', 
    data=data, 
    ci=False, 
)

- `hue` means _color_, but it also causes _seaborn_ to fit a different linear model to each of the Classes. 
- `ci` controls the confidence intervals. I chose `False`, but setting it to `True` will show them.

We can also seperate each class to a different plot (with the `col` argument):

In [ ]:
sns.lmplot(
    x='Body mass (kg)', 
    y='Metabolic rate (W)', 
    hue='Class', 
    col = 'Class',
    data=data, 
    ci=True, 
);

We can see that mammals and birds have a clear correlation between size and metabolism and that it extends over a nice range of mass.

So let's stick to mammals; next up we will see which orders of mammals we have.

In [ ]:
mammalia = data[data.Class=='Mammalia']
order_counts = mammalia['Order'].value_counts()
ax = order_counts.plot.barh()
ax.set(
    xlabel='Num. of species',
    ylabel='Mammalia order'
)
ax.figure.set_figheight(7)

You see we have alot of rodents and carnivores, but also a good number of bats (_Chiroptera_) and primates.

Let's continue with orders that have at least 20 species - this also includes some cool marsupials like Kangaroo, Koala and [Taz](http://upload.wikimedia.org/wikipedia/en/c/c4/Taz-Looney_Tunes.svg) (Diprotodontia and Dasyuromorphia)

In [ ]:
orders = order_counts[order_counts >= 20]
print(orders)
abund_mammalia = mammalia[mammalia['Order'].isin(orders.index)]

In [ ]:
sns.lmplot(
    x='Body mass (kg)', 
    y='Metabolic rate (W)', 
    hue='Order',
    data=abund_mammalia, 
    ci=False, 
    height=8,
    aspect=1.3,
    line_kws={'lw':2, 'ls':'--'}, 
    scatter_kws={'s':50, 'alpha':0.5}
);
# if you get an error about height not being a keyword, change it to size or update seaborn: conda update seaborn

Because there is alot of data here we made the lines thinner - this can be done by giving _matplotlib_ keywords as a dictionary to the argument `line_kws` - and we made the markers bigger but with alpha (transperancy) 0.5 using the `scatter_kws` argument.

Still ,there's too much data, and part of the problem is that some orders are large (e.g. primates) and some are small (e.g. rodents).

Let's plot a separate regression plot for each order.
We do this using the `col` and `row` arguments of `lmplot`, but in general this can be done for any plot using [seaborn's `FacetGrid` function](http://stanford.edu/~mwaskom/software/seaborn/tutorial/axis_grids.html).

In [ ]:
sns.lmplot(
    x='Body mass (kg)', 
    y='Metabolic rate (W)', 
    data=abund_mammalia, 
    hue='Order',
    col='Order', 
    col_wrap=3, 
    ci=False, 
    scatter_kws={'s':40}, 
    sharex=False, 
    sharey=False
);

We used the `sharex=False` and `sharey=False` arguments so that each Order will have a different axis range and so the data is spread nicely.

There are plenty more types of figures, look at [seaborn galery](https://seaborn.pydata.org/examples/index.html).

One more example - [jointplot](https://seaborn.pydata.org/examples/hexbin_marginals.html):

In [ ]:
small = mammalia[mammalia['Body mass (kg)'] < 1]
sns.jointplot(
    x='Body mass (kg)', 
    y='Metabolic rate (W)', 
    data=small, 
    kind='reg'
);

## __Some statistics__

Lastly, let's do some quick statistics.

First, calculate a summary of the the mammals using `describe`.

In [ ]:
mass = abund_mammalia
mass.describe()

Now lets check if we can significantly say that the body mass of rodents is lower than that of carnivores.

## Exercise: boxplot
**Plot boxplots of the mammals body mass** using Seaborn, which is easier to use (and also makes nicer boxplots) then standard matplotlib boxplot.

In [ ]:
# data: abund_mammalia

Now, we'll use a t-test (implemented in the `scipy.stats` module) to test the hypothesis that there is *no difference* in body mass between rodents and carnivores.

- `ttest_ind` calculates the t-test for the means of *two independent* samples of scores.
- `scipy.stats` has many more statistical tests, distributions, etc.

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
carnivora_mass = abund_mammalia.loc[abund_mammalia['Order']=='Carnivora', 'Body mass (kg)']
rodentia_mass = abund_mammalia.loc[abund_mammalia['Order']=='Rodentia', 'Body mass (kg)']

res = ttest_ind(carnivora_mass, rodentia_mass, equal_var=False)
print(res)
print("P-value of t-test: {:.3g}".format(res.pvalue))

## __Some more data manipulatoin__

In this section we will analyze a data regarding machines malfunctioning.

For each machine, the data shows after how many hours it got broken, after how many hours a decision regarding the machine's status has been made, and the final status of the machine - whether it was fixed or sent for scrapping.

For each machine we have the following parameters:
* model
* submodel
* quality
* type
* stop (after how many hours the machine got broken)
* diagonosis (after how many hours a decision regarding the machine has been made - either it was fixed or sent for scrapping)
* fix-scrapping (0 means the machine was demolished, 1 means it was fixed)

Let's look at the data:

In [ ]:
data = pd.read_csv(r'..\data\sim_data\sim_part1.csv')
print(data.shape)
data.head()

But we have 3 csv files: sim_part1, sim_part2, sim_part3

We need to load them all, and merge:

In [ ]:
import os

folder = r'..\data\sim_data'
files = os.listdir(folder)

datas = [pd.read_csv(folder + '\\' + files[i]) for i in range(len(files))]

# concatenate all datasets
data = pd.concat(datas, axis = 0)

print(data.shape)
data.head()

Let's see how many unique parameter values we have:

In [ ]:
for param in data.columns[:4]: # run over the first 4 column names
    print(param, '-', sorted(data[param].unique()))

We can see that all machines have the same model and submodel.

So we can slice the data and focus on the parameters that do change:

In [ ]:
data2 = data.drop(columns=['model', 'submodel']) # add: inplace=True if you wish to change the original data
print(data2.shape)
data2.head()

We would like to add a column, that contains the duration of the fixing time (the diagnosis minus the stop).

Note that in our records some processes ended with fixing (fix-scrapping = 1), and some with scrapping (fix-scrapping = 0).

## Exercise

Add a column to `data` that contains, for each row, the `diagnosis` value, minus the `stop` value.

For lines where fix-scrapping = 1, we want this subtraction result,

but for lines where fix-scrapping = 0, we want the value in the column to be zero. 

Call the new column `fixtime`.

In [ ]:
data2['fixtime'] = 

data2.head(11)

We are interested in the fixtime only when the process finished with fixing (fix-scrapping=1).

Currently, if for example we would want to get the average fixtime - if we will average the column we will have many 0 values.

Possible solution - change 0 values to None:

In [ ]:
print(data2.fixtime[:12].mean())

In [ ]:
data2.loc[data2['fix-scrapping'] == 0, 'fixtime'] = np.nan
print(data2.fixtime[:12].mean())
data2.head(11)

We can now remove `diagnosis` column as we no longer need it:

In [ ]:
data2.drop(columns=['diagnosis'], inplace=True)
# The `inplace=True` command makes the function remove the column from the current DF and not make a new copy of the DF 
data2.head()

Now we would like to examine the effect of `quality`, `type` over the rest of the parameters. 

In order to do so we would like to extract all kinds of summary statistics for each `quality`, `type` couple.

In order to do so we first need to state that we want to group our data according to `quality` and `type`.

And then we need to aggregate the results with the desired operation (e.g. mean, variance, std etc.).

In [ ]:
grouped = data2.groupby(['quality', 'type'])
aggdata = grouped.agg(np.mean)

print(aggdata.shape)
aggdata

What if we want more than one statistics? For example mean, std and len (= # of observations).

We can specify for each column what are the operators we want to activate.

We can do this using pandas [agg](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) method. The input of the method is a dictionary where: 
* the keys are the column names
* the values are lists of functions/methods.

In [ ]:
aggdata = grouped.agg({'stop' : [np.mean, np.std, len], \
                       'fixtime' : [np.mean, np.std, len], \
                       'fix-scrapping' : [np.mean, np.std, len]})
aggdata

In 'fixtime' column we wanted to consider only non-NaN values. 

np.mean and np.std indeed didn’t consider NaN values but len function did!

In order to overcome this, we’ll define our own function:

In [ ]:
def f(x): # x is an array of values
    '''this function returns the size of the array, 
    minus the number of nan values in that array'''
    return (~np.isnan(x)).sum() # ~ is elementwise not

In [ ]:
aggdata = grouped.agg({'stop' : [np.mean, np.std, len],
                       'fixtime' : [np.mean, np.std, f], # here we also set the names of the columns
                       'fix-scrapping' : [np.mean, np.std, len]})
aggdata

In [ ]:
aggdata = grouped.agg({'stop' : [np.mean, np.std, len],
                       'fixtime' : [('mean',np.mean), ('std',np.std), ('len',f)], # here we also set the names of the columns
                       'fix-scrapping' : [np.mean, np.std, len]})
aggdata

In [ ]:
aggdata.reset_index(inplace=True)
aggdata

We can now present our data in a figure. We’ll make a 3-panels figure:
* stop time
* fixing time
* fixing probability

All three as a function of the quality parameter.

Each panel will show two curves, one for type A and another for type B.

We start by generating the formation of our figure:

```py
fig, ax = plt.subplots(1, 3, figsize = (16,5))
```

`plt.subplots` return 2 objects
* A figure object
* An array of axes  objects 

In [ ]:
panels = ['stop', 'fixtime', 'fix-scrapping']
ylabels = ['Time to break', 'Fixing time', 'Fixing probability'] ###<->### for easy ylabels settings

typeValues = ['A','B']
xvec = sorted(data2['quality'].unique()) # generate x-axis

fig, ax = plt.subplots(1, 3, figsize = (16,6))

for i in range(3):  # in each iteration we’ll produce a different subplot
    param = panels[i]
    for j in range(2): # in each iteration we’ll produce a different curve
        tval = typeValues[j]
        
        yvec = aggdata[aggdata['type'] == tval][param]['mean']
        curdata_len = aggdata[aggdata['type'] == tval][param]['len']
        ste = aggdata[aggdata['type'] == tval][param]['std'] / np.sqrt(curdata_len)
        ax[i].errorbar(xvec, yvec, yerr = ste, fmt = '--o', label= 'type = '+tval) ###<->###
    
    ax[i].set_ylabel(ylabels[i], fontsize = 25)
    ax[i].set_xlabel('quality', fontsize = 25)
    ax[i].tick_params(labelsize = 20) # set tick labels size
    ax[i].set_xticks(np.arange(6))
    ax[i].set_xticklabels(np.arange(6))
    if i == 2: ax[i].legend(fontsize=20, loc=0) # legend only in right panel
    ax[i].set_title('ABC'[i], fontsize=30, fontweight='bold', y=1.07, x= -0.1)

fig.tight_layout() # adjust the space between the panels

# References

- Examples: [Seaborn example gallery](http://seaborn.pydata.org/examples/index.html)
- Slides: [Statistical inference with Python](https://docs.google.com/presentation/d/1imQAEmNg4GB3bCAblauMOOLlAC95-XvkTSKB1_dB3Tg/pub?slide=id.p) by Allen Downey
- Book: [Think Stats](greenteapress.com/thinkstats2/html/index.html) by Allen Downey - statistics with Python. Free Ebook.
- Blog post: [A modern guide to getting started with Data Science and Python](http://twiecki.github.io/blog/2014/11/18/python-for-data-science/)
- Tutorial: [An Introduction to Pandas](http://www.synesthesiam.com/posts/an-introduction-to-pandas.html)

# Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

The notebook was written using [Python](http://python.org/) 3.7.
Dependencies listed in [environment.yml](../environment.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)